In [1]:
!pip install numpy
!pip install torch
!pip install -U scikit-learn
!pip install panda



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\CCL\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\CCL\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\CCL\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for panda: filename=panda-0.3.1-py3-none-any.whl size=7245 sha256=204df6e79efb22b4d1cbfcabb5b7a101058f5a52d58aa0b2dbe800a13a6a7680
  Stored in directory: c:\users\ccl\appdata\local\pip\cache\wheels\df\5c\39\36f8dae25a1e88d6ec4411dec4a143781e64fdff6897758eec
Successfully built panda



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\CCL\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from giza_datasets import DatasetsHub, DatasetsLoader
import os
import certifi

loader = DatasetsLoader()

# Neural Network Definition
input_size = 6  # Number of input features
hidden_size1 = 64
hidden_size2 = 32
output_size = 1
learning_rate = 0.001
batch_size = 128
num_epochs = 50

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        return out

# Instantiate the Neural Network
model = NeuralNet(input_size=input_size, hidden_size1=hidden_size1, hidden_size2=hidden_size2, output_size=output_size)

# Move model to device
model.to(device)

# Define Loss function and Optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Load the dataset using Polar
df_polar = loader.load('yearn-individual-deposits')

# Convert the Polar DataFrame to a Pandas DataFrame
df = df_polar.to_pandas()

# Extract features and target variable
features = df[['evt_block_number', 'token_decimals', 'evt_block_time']]  # Include 'evt_block_time' in features
target = df['value']

# Convert 'evt_block_time' to pandas datetime format
features['evt_block_time'] = pd.to_datetime(features['evt_block_time'])

# Extract time-based features
features['hour_of_day'] = features['evt_block_time'].dt.hour
features['day_of_week'] = features['evt_block_time'].dt.dayofweek
features['month'] = features['evt_block_time'].dt.month

# Drop 'evt_block_time' as it's no longer needed
features = features.drop(columns=['evt_block_time'])

    # Define batch size
batch_size = 256

# Encode 'token_contract_address' as categorical
unique_addresses = sorted(set(df['token_contract_address']))
address_to_label = {addr: i for i, addr in enumerate(unique_addresses)}
encoded_addresses = [address_to_label[addr] for addr in df['token_contract_address']]

# Create a new DataFrame with the encoded 'token_contract_address' column
features['token_contract_address'] = encoded_addresses

# Apply min-max scaling to the features
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features.values)
features = pd.DataFrame(scaled_features, columns=features.columns)

# Convert DataFrame to PyTorch tensors
x_data = torch.tensor(features.values.astype("float32"))
y_data = torch.tensor(target.to_numpy().astype("float32"))

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)


train_loader = DataLoader(
    TensorDataset(x_train, y_train), batch_size=batch_size, shuffle=True
    )

train_loader = DataLoader(
    TensorDataset(x_test, y_test), batch_size=batch_size, shuffle=True
    )

def train_model(train_loader, model, criterion, optimizer, num_epochs, device):
    """
    Train the neural network model.

    Args:
        train_loader (DataLoader): DataLoader containing the training data.
        model (NeuralNet): Neural network model to be trained.
        criterion (torch.nn._Loss): Loss function.
        optimizer (torch.optim.Optimizer): Optimization algorithm.
        num_epochs (int): Number of epochs for training.
        device (torch.device): Device to train the model on (CPU or GPU).

    Returns:
        model (NeuralNet): Trained neural network model.
    """
    print("Train model...")

    # Move model to device
    model.to(device)

    # Iterate over epochs
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0

        # Iterate over batches in the training data
        for i, (data, labels) in enumerate(train_loader):
            # Move data to device
            data = data.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(data)

            # Calculate the loss
            loss = criterion(outputs, labels)

            # Backpropagation and optimization step
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # Print training progress
            if (i + 1) % 100 == 0:
                print(
                    f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / 100:.4f}"
                )
                running_loss = 0.0

    print("✅ Model trained successfully")
    return model


train_model(train_loader, model, criterion, optimizer, num_epochs, device)



C:\Users\CCL\AppData\Local\Temp\ipykernel_17140\1350151848.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['evt_block_time'] = pd.to_datetime(features['evt_block_time'])
C:\Users\CCL\AppData\Local\Temp\ipykernel_17140\1350151848.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['hour_of_day'] = features['evt_block_time'].dt.hour
C:\Users\CCL\AppData\Local\Temp\ipykernel_17140\1350151848.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

Train model...


C:\Users\CCL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([256])) that is different to the input size (torch.Size([256, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\CCL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([127])) that is different to the input size (torch.Size([127, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


✅ Model trained successfully


NeuralNet(
  (fc1): Linear(in_features=6, out_features=64, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=32, out_features=1, bias=True)
)

In [5]:
from sklearn.metrics import precision_score, recall_score, f1_score

def test_model(model, test_loader, threshold=0.5):
    print("Test model...")
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        y_true = []
        y_pred = []
        for data, labels in test_loader:
            # Move data to device and reshape if necessary
            data = data.to(device)
            if len(data.shape) > 2:  # If data is not flattened
                data = data.view(data.size(0), -1)
            labels = labels.to(device)
            # Forward pass
            outputs = model(data)
            predicted_probs = torch.sigmoid(outputs)
            predicted_labels = (predicted_probs > threshold).float()
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted_labels.cpu().numpy())

        y_true_binary = [1 if label > threshold else 0 for label in y_true]
        y_pred_binary = [1 if label > threshold else 0 for label in y_pred]

        precision = precision_score(y_true_binary, y_pred_binary)
        recall = recall_score(y_true_binary, y_pred_binary)
        f1 = f1_score(y_true_binary, y_pred_binary)

        print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

    return precision, recall, f1

# Assuming you have already defined the `train_loader` and `model`
precision, recall, f1 = test_model(model, train_loader)


Test model...
Precision: 0.8145, Recall: 1.0000, F1-score: 0.8978
